下载好模型
pip install -U huggingface_hub


export HF_ENDPOINT=https://hf-mirror.com

huggingface-cli download --resume-download openai/clip-vit-large-patch14-336 --local-dir openai/clip-vit-large-patch14-336 --local-dir-use-symlinks False

 
huggingface-cli download --resume-download Qwen/Qwen1.5-4B-Chat --local-dir Qwen1.5-4B-Chat --local-dir-use-symlinks False 
0.5/1.8/4/7B

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

modify_qwen_tokenizer_dir = "Qwen1.5-4B-Chat"
modify_qwen_tokenizer = AutoTokenizer.from_pretrained(modify_qwen_tokenizer_dir)

modify_qwen_tokenizer.encode("<image>")

#此处的占位符有三个维度，我们要修改到一个维度 [27, 1805, 29]

In [ ]:
len(modify_qwen_tokenizer)
#151647

Q: 加了这个新token，需要修改模型的embedding模块么？ A：不需要，qwen_model.model.embed_tokens留了足够的空间

In [ ]:
qwen_model=AutoModelForCausalLM.from_pretrained(
    modify_qwen_tokenizer_dir,device_map="cuda:1",torch_dtype=torch.bfloat16)


In [ ]:
qwen_model.model.embed_tokens
#Embedding(151936, 2048)

In [ ]:
# 1. Tokenizer 负责把文字变成 ID
text = "你好"
input_ids = modify_qwen_tokenizer.encode(text, return_tensors='pt')

print("1. 文字:", text)
print("2. Tokenizer 翻译出的 ID:", input_ids) 
# 结果可能像: tensor([[108386]])

# 2. Embed_tokens 负责把 ID 变成向量
# 注意：我们要把 ID 扔给 embed_tokens 层
# (如果模型在 CPU，这里 input_ids 也在 CPU 就行)
vectors = qwen_model.model.embed_tokens(input_ids)

print("3. 最终查到的向量形状:", vectors.shape)
# 结果应该是: torch.Size([1, 1, 2048]) -> [Batch, 序列长度, 隐藏层维度]
# 这说明"你好"这个词，被转换成了长度为 2048 的一串数字。

In [ ]:
qwen_model.lm_head

embed_tokens：字→向量。让计算机读懂输入。  
lm_head：向量->字的概率。让计算机生成输出。  
它们就像的一座大桥的桥头和桥尾，中间的桥身就是那一堆 Transformer 层  

In [ ]:
clip_model_name_or_path = (
    "openai/clip-vit-large-patch14-336"
)
qwen_model_name_or_path = "Qwen1.5-4B-Chat"

In [ ]:
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, AutoProcessor

clip_model = AutoModel.from_pretrained(clip_model_name_or_path, device_map="cuda:1")
llm_model = AutoModelForCausalLM.from_pretrained(
    qwen_model_name_or_path, device_map="cuda:1"
)

In [ ]:
llm_tokenizer = AutoTokenizer.from_pretrained(qwen_model_name_or_path)
llm_tokenizer.encode("<image>")

初始化llava
将clip模型和llm_model模型的config拿出来，初始化一个llava model

In [ ]:
from transformers import (
    LlavaForConditionalGeneration,
    LlavaConfig
)

In [ ]:
# Initializing a CLIP-vision config
vision_config = clip_model.vision_model.config

# Initializing a Llama config
text_config = llm_model.config

# Initializing a Llava llava-1.5-7b style configuration
configuration = LlavaConfig(vision_config, text_config)

# Initializing a model from the llava-1.5-7b style configuration
model = LlavaForConditionalGeneration(configuration)

In [ ]:
model.vision_tower.vision_model.embeddings

In [ ]:
clip_model.vision_model

形状初始化好了，模型权重都还是随机生成的，需要把两个模型的权重，复制过去

In [ ]:
model.vision_tower.vision_model = clip_model.vision_model
model.language_model = llm_model

In [ ]:
model.config.pad_token_id

In [ ]:
#把qwen tokenizer分词器里定义的填充 token 编号同步给模型配置
model.config.pad_token_id = llm_tokenizer.pad_token_id
model.config.pad_token_id

In [ ]:
model.config.image_token_index

In [ ]:
llm_tokenizer.encode("<image>")[0]#这个值需要被替换

In [ ]:
model.config.image_token_index = llm_tokenizer.encode("<image>")[0]
model.config.image_token_index

In [ ]:
model.save_pretrained("show_model_v1/model001")

In [ ]:
llm_tokenizer.save_pretrained("show_model_v1/model001")

In [ ]:
autoprocessor = AutoProcessor.from_pretrained(clip_model_name_or_path)
autoprocessor.save_pretrained("show_model_v1/model002")

主要需要把show_model/model002里面的preprocessor_config.json文件，放在show_model/model001里面

重启内核

In [1]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import torch
from PIL import Image

model_name_or_path = "show_model_v1/model001"  # 
# model_name_or_path = "test_model_copy/model001"  #

processor = AutoProcessor.from_pretrained(model_name_or_path)
model = LlavaForConditionalGeneration.from_pretrained(
    model_name_or_path, device_map="cuda:1", torch_dtype=torch.bfloat16
).eval()

/home/ZJH/miniconda3/envs/mote/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Qwen2Tokenizer'. 
The class this function is called from is 'CLIPTokenizerFast'.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.16s/it]


In [2]:
testdata = (
'<image>\nRelay a brief, clear account of the picture shown.', # 提问
 'a big mountain with a lake', # 真实答案
 'R.jpeg' # 图片路径
)


In [5]:
def build_model_input(model, processor, testdata:tuple):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": testdata[0]},
    ]
    prompt = processor.tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    # print(prompt)
    # print("*"*20)
    image = Image.open(testdata[2])
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    
    for tk in inputs.keys():
        if tk == "pixel_values":
            inputs[tk] = inputs[tk].to(model.device, dtype=torch.bfloat16)
        else:
            inputs[tk] = inputs[tk].to(model.device)
    generate_ids = model.generate(**inputs, max_new_tokens=20)
    
    generate_ids = [
        oid[len(iids):] for oid, iids in zip(generate_ids, inputs.input_ids)
    ]

    

    gen_text = processor.batch_decode(generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]
    return gen_text,inputs


In [6]:
result, debug_inputs = build_model_input(model, processor, testdata)

# 'the kitchen is a bright yellow with a glass top island and a large window that looks out to the'


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [7]:
print(result)

I ' m   s o r r y ,   b u t   I   c a n n o t   p r o v i d e   a   p i c t u r e   a s   I   a m   a n   A I   l a n g u a g e   m o d e l   a n d   d o   n o t


In [8]:
# 查看 inputs 包含哪些键
print(debug_inputs.keys())

# 查看 input_ids 的形状
print(debug_inputs.input_ids.shape)

dict_keys(['input_ids', 'attention_mask', 'pixel_values'])
torch.Size([1, 33])


In [9]:
model.config

LlavaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "show_model_v1/model001",
  "architectures": [
    "LlavaForConditionalGeneration"
  ],
  "ignore_index": -100,
  "image_seq_length": 576,
  "image_token_index": 151646,
  "model_type": "llava",
  "pad_token_id": 151643,
  "projector_hidden_act": "gelu",
  "text_config": {
    "_attn_implementation_autoset": true,
    "_name_or_path": "Qwen1.5-4B-Chat",
    "add_cross_attention": false,
    "architectures": [
      "Qwen2ForCausalLM"
    ],
    "attention_dropout": 0.0,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": 151643,
    "chunk_size_feed_forward": 0,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 151645,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id":